In [1]:
from PIL import Image
import os
import pandas as pd
import glob
import numpy as np
import shutil
from tqdm.auto import tqdm
from utility import  parse_xml,check_yolo_label

In [2]:
OUTPUT_SIZE = 512
ADDITIONAL_CLEARANCE = 15
ADDITIONAL_CLEARANCE_SMALL_DRONE = 15
NUMBER_OUTPUT_IMG = 1000000
MIN_AREA_DRONE = 0.03
MAX_AREA_DRONE = 0.05
AREA_SMALL_DRONE =  0.1

DATASET_DIR = r"C:\project\Computer_Vision\term_paper_2023\datasets\antiuav_train"
IMAGE_INPUT_DIR = r"C:\project\Computer_Vision\term_paper_2023\datasets\antiuav_train\img"
LABEL_INPUT_DIR = r"C:\project\Computer_Vision\term_paper_2023\datasets\antiuav_train\xml"

OUTPUT_DIR = r"C:\project\Computer_Vision\term_paper_2023\antiuav_with_masks"
IMAGE_OUTPUT_DIR = r"C:\project\Computer_Vision\term_paper_2023\antiuav_with_masks\images"
LABEL_OUTPUT_DIR = r"C:\project\Computer_Vision\term_paper_2023\antiuav_with_masks\labels"
MASK_OUTPUT_DIR = r"C:\project\Computer_Vision\term_paper_2023\antiuav_with_masks\masks"

shutil.rmtree(OUTPUT_DIR)
os.makedirs(os.path.join(IMAGE_OUTPUT_DIR))
os.makedirs(os.path.join(LABEL_OUTPUT_DIR))
os.makedirs(MASK_OUTPUT_DIR)


def get_dron_df():
    df = pd.DataFrame (glob.glob( os.path.join(IMAGE_INPUT_DIR,'*.jpg')), columns=['path_image'])
    df['id'] = df['path_image'].apply(lambda x : x.split('\\')[-1].split('.')[0])
    df['path_label'] = df['id'].apply(lambda x: os.path.join(LABEL_INPUT_DIR,f'{x}.xml'))
    return df

df_drons = get_dron_df()
dron_statistics = []
mask_statistics = []
id_statistics = []
additional_clearance_statistics = []

In [3]:

for i, row in tqdm(df_drons.sample(frac=1)[:NUMBER_OUTPUT_IMG].iterrows()):

    coords_drons, width_image, height_image = parse_xml(row['path_label'])

    if len(coords_drons) == 0:
        continue;

    xmin, ymin, xmax, ymax = coords_drons[0]

    drone_width_fraction = (xmax - xmin) / width_image
    drone_height_fraction = (ymax - ymin) / height_image
    drone_area_fraction = drone_width_fraction*drone_height_fraction
    if  drone_height_fraction > MAX_AREA_DRONE or drone_height_fraction < MIN_AREA_DRONE:
        continue

    #Новые координаты
    drone_width = int(drone_width_fraction*OUTPUT_SIZE)
    drone_height = int(drone_height_fraction*OUTPUT_SIZE)
    drone_width += 2 * ADDITIONAL_CLEARANCE_SMALL_DRONE
    drone_height += 2 * ADDITIONAL_CLEARANCE_SMALL_DRONE
    drone_xmin = int(xmin/width_image*OUTPUT_SIZE) - ADDITIONAL_CLEARANCE_SMALL_DRONE
    drone_ymin = int(ymin/height_image*OUTPUT_SIZE) - ADDITIONAL_CLEARANCE_SMALL_DRONE
    drone_xmax =  drone_xmin+drone_width
    drone_ymax =  drone_ymin+drone_height

    image = Image.open(row['path_image']).resize((OUTPUT_SIZE, OUTPUT_SIZE))
    drone = image.crop((drone_xmin, drone_ymin, drone_xmax, drone_ymax))

    #Смещение
    drone_xmin = np.random.randint(0, OUTPUT_SIZE - drone_width, 1)[0]
    drone_ymin = np.random.randint(0, OUTPUT_SIZE - drone_height, 1)[0]
    drone_xmax =  drone_xmin+drone_width
    drone_ymax =  drone_ymin+drone_height

    mask = Image.new("RGBA", (OUTPUT_SIZE, OUTPUT_SIZE), (255, 255, 255, 255))
    rect = Image.new("RGBA",  (drone_width, drone_height), (0, 0, 0, 255))
    mask.paste(rect, (drone_xmin, drone_ymin))
    # image.show()
    image = Image.new("RGB", (OUTPUT_SIZE, OUTPUT_SIZE), (255, 255, 255))
    image.paste(drone, (drone_xmin, drone_ymin))
    # drone.show()
    # image.show()
    # mask.show()
    # break
    image.save(os.path.join(IMAGE_OUTPUT_DIR, row['id'] + '.jpg'))
    mask.save(os.path.join(MASK_OUTPUT_DIR, f"{row['id']}.png"))

    x_center_yolo = (drone_xmin + drone_width / 2) / OUTPUT_SIZE
    y_center_yolo = (drone_ymin + drone_height / 2) / OUTPUT_SIZE
    str_label_yolo = f"0 {x_center_yolo} {y_center_yolo} {drone_width_fraction} {drone_height_fraction}  "
    # check_yolo_label(image, x_center_yolo, y_center_yolo, drone_width_fraction, drone_height_fraction )
    path_label_output = os.path.join(LABEL_OUTPUT_DIR, row['id'] + '.txt')
    with open(path_label_output, 'w') as f:
            f.write(str_label_yolo)

    additional_clearance_statistics.append(ADDITIONAL_CLEARANCE_SMALL_DRONE)
    dron_statistics.append(drone_area_fraction)
    id_statistics.append(row['id'])
    mask_statistics.append(drone_area_fraction/OUTPUT_SIZE/OUTPUT_SIZE)


df_statistics = pd.DataFrame({'id': id_statistics,
                              'dron': dron_statistics,
                              'mask' : mask_statistics,
                              'additional clearance': additional_clearance_statistics
                              })
df_statistics.to_csv(os.path.join(OUTPUT_DIR, "statistics.csv"))

0it [00:00, ?it/s]